In [1]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

In [2]:
S_b = 90e3
U_b = 400.0
Z_b = U_b**2/S_b
I_b = S_b/(np.sqrt(3)*U_b)
Omega_b = 2*np.pi*50
R_s = 0.023/Z_b
R_r = 0.024/Z_b
Ll_s = 0.086/Z_b
Ll_r = 0.196/Z_b
L_m = 3.7/Z_b


params = {'S_b':S_b,'U_b':U_b,'I_b':I_b,
          'R_s':R_s,'R_r':R_r,'L_ls':Ll_s,'L_lr':Ll_r,'L_m':L_m,   # synnchronous machine d-axis parameters
          'H_m':3.5,'Omega_b':2*np.pi*50,'D':0.1,
          'v_0':1,'theta_0':0.0,
          'X_l':0.05, 'omega_s':1.0,'v_rd':0.0,'v_rq':0.0}



u_ini_dict = {'P_h':0.1,'Q_h':0.0}  # for the initialization problem
u_run_dict = {'tau_m':0.1,'B_c':0.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['omega_e','psi_rd','psi_rq']    # [inductor current, PI integrator]
y_ini_list = ['psi_sd','psi_sq','i_sd','i_sq','i_rd','i_rq','tau_m','B_c','v_h','theta_h'] # for the initialization problem
y_run_list = ['psi_sd','psi_sq','i_sd','i_sq','i_rd','i_rq','P_h','Q_h','v_h','theta_h'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
v_sd =  v_h*sin(-theta_h) 
v_sq =  v_h*cos(-theta_h) 

tau_e = psi_sd*i_sq - psi_sq*i_sd


domega_e = 1/(2*H_m)*(tau_m - tau_e - D*omega_e)

dpsi_rd = Omega_b*(-v_rd -R_r*i_rd - (omega_s-omega_e)*psi_rq)
dpsi_rq = Omega_b*(-v_rq -R_r*i_rq + (omega_s-omega_e)*psi_rd)

g_psi_sd = Omega_b*(-v_sd -R_s*i_sd - omega_s*psi_sq)
g_psi_sq = Omega_b*(-v_sq -R_s*i_sq + omega_s*psi_sd)
g_1 = -psi_sd + (L_m + L_ls)*i_sd + L_m*i_rd
g_2 = -psi_sq + (L_m + L_ls)*i_sq + L_m*i_rq
g_3 = -psi_rd + (L_m + L_lr)*i_rd + L_m*i_sd
g_4 = -psi_rq + (L_m + L_lr)*i_rq + L_m*i_sq
        
Q_c = B_c*v_h**2

g_5 = P_h - (v_h*v_0*sin(theta_h - theta_0))/X_l
g_6 = Q_h + (v_h*v_0*cos(theta_h - theta_0))/X_l - v_h**2/X_l
g_7 = -P_h + (v_sd*i_sd + v_sq*i_sq)
g_8 = -Q_h + (v_sd*i_sq - v_sq*i_sd) + Q_c

h_1 = I_b*(i_sd*i_sd + i_sq*i_sq)**0.5
h_p = v_sd
h_q = v_sq
h_qc = Q_c

sys = {'name':'imib_fisix_3ord',
       'params':params,
       'f':[domega_e,dpsi_rd,dpsi_rq],
       'g':    [g_psi_sd,g_psi_sq,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8],
       'g_ini':[g_psi_sd,g_psi_sq,g_1,g_2,g_3,g_4,g_5,g_6,g_7,g_8],
       'x':x_list,
       'y_ini':y_ini_list,
       'y':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h':[h_1,h_p,h_q,h_qc]}

sys = db.system(sys)
db.sys2num(sys)

In [ ]:
sys['f']